In [2]:
import os 
import sys
import time
import pandas as pd
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [8]:
class MonitorFolder(FileSystemEventHandler):
    """
    Class for monitoring any changes in folder of the csv file 

    Attributes :
        MAX_SIZE (int) : Size threshold in bytes for the folder or file for triggerring parallelization

    Methods :

        on_created (event) : Prints message and calls checkFolderSize to check size of the folder when folder is created
        on_modified (event) : Prints a message and calls checkFolderSize to check size of the folder when folder is created
        on_deleted(event) : Prints a message when a file or folder is deleted
        checkFolderSize(src_path) : Checks the size of a file or folder and prints a message if the size exceeds MAX_SIZE

    https://medium.com/analytics-vidhya/monitoring-your-file-system-using-watchdog-64f7ad3279f

    """

    MAX_SIZE = 1000

    def checkFolderSize(self, src_path):
        if os.path.getsize(src_path) > self.MAX_SIZE:
            print("Directory getting big")
        else : 
            if os.path.getsize(src_path) > self.MAX_SIZE:
                print("Very big file, commencing parallelization")


    def on_created(self, event):
        print(f"{event.src_path} was created")
        data = pd.read_csv(event.src_path + "\\AAPL_prices.csv")
        self.checkFolderSize(event.src_path)
        print(data)

    def on_modified(self, event):
        print(f"{event.src_path} was modified")
        self.checkFolderSize(event.src_path + "\\AAPL_prices.csv")
        data = pd.read_csv(event.src_path)

    def on_deleted(self, event):
        print(f"{event.src_path} was deleted")

if __name__ == "__main__":
    src_path = "C:\\Users\\lilib\\Documents\\Cours M2\\Big Data\\Data_Streaming_tse"
    
    event_handler=MonitorFolder()
    observer = Observer()
    observer.schedule(event_handler, path=src_path, recursive=True)
    print("Monitoring started")
    observer.start()
    try:
        while(True):
           time.sleep(1)
           
    except KeyboardInterrupt:
            observer.stop()
            observer.join()

Monitoring started


Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\Users\lilib\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\lilib\AppData\Local\Programs\Python\Python311\Lib\site-packages\watchdog\observers\api.py", line 204, in run
    self.dispatch_events(self.event_queue)
  File "c:\Users\lilib\AppData\Local\Programs\Python\Python311\Lib\site-packages\watchdog\observers\api.py", line 380, in dispatch_events
    handler.dispatch(event)
  File "c:\Users\lilib\AppData\Local\Programs\Python\Python311\Lib\site-packages\watchdog\events.py", line 276, in dispatch
    {
  File "C:\Users\lilib\AppData\Local\Temp\ipykernel_13388\3974939611.py", line 37, in on_modified
  File "C:\Users\lilib\AppData\Local\Temp\ipykernel_13388\3974939611.py", line 22, in checkFolderSize
  File "<frozen genericpath>", line 50, in getsize
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\lili

C:\Users\lilib\Documents\Cours M2\Big Data\Data_Streaming_tse\AAPL_prices.csv was modified
